In [ ]:
import matplotlib
# matplotlib.use("Agg")
import keras
from keras.callbacks import ModelCheckpoint
# import the necessary packages
from tensorflow import keras
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.layers import concatenate
from keras.models import Model
from keras.optimizers import Optimizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import load_model
from keras import regularizers
from imutils import paths
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import argparse
import pickle
import cv2
import os
from PIL import Image
import hickle
from os import listdir 
from os.path import isfile, join, isdir
import random
import sys
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
data_att ='/media/senilab/DATA2/new_feature_att_hmdb/feature_rgb_train_i3d_5c_64_att_new_split1_att128'
load_att_rgb = hickle.load(data_att)
input_feat_att_rgb = load_att_rgb['feat']
input_label_att_rgb = load_att_rgb['label']

data_att ='/media/senilab/DATA2/new_feature_att_hmdb/feature_flow_train_i3d_5c_64_att_new_split1_att128'
load_att = hickle.load(data_att)
input_feat_att = load_att['feat']
input_label_att = load_att['label']

In [ ]:
data_att ='/media/senilab/DATA2/new_feature_att_hmdb/feature_rgb_test_i3d_5c_64_att_new_split1_att128'
load_att_rgb = hickle.load(data_att)
test_feat_att_rgb = load_att_rgb['feat']
test_label_att_rgb = load_att_rgb['label']

data_att ='/media/senilab/DATA2/new_feature_att_hmdb/feature_flow_test_i3d_5c_64_att_new_split1_att128'
load_att = hickle.load(data_att)
test_feat_att = load_att['feat']
test_label_att = load_att['label']

In [ ]:
import keras

input1 = keras.layers.Input(shape=(64,1024))
x1 = Flatten()(input1)
x1 = keras.layers.Dropout(0.9)(x1)
# x1 = keras.layers.Dense(64, input_dim=64,kernel_regularizer=regularizers.l2(0.001),activity_regularizer=regularizers.l1(0.001))(x1)

input2 = keras.layers.Input(shape=(64,1024))
x2 = Flatten()(input2)
x2 = keras.layers.Dropout(0.9)(x2)
# x2 = keras.layers.Dense(64, input_dim=64,kernel_regularizer=regularizers.l2(0.001),activity_regularizer=regularizers.l1(0.001))(x2)

# equivalent to added = keras.layers.add([x1, x2])
added = keras.layers.Add()([x1, x2])
# added = keras.layers.Dropout(0.9)(added)
added = keras.layers.Dense(64, input_dim=64,kernel_regularizer=regularizers.l2(0.001),activity_regularizer=regularizers.l1(0.001))(added)

out = keras.layers.Dense(51, activation='softmax')(added)
model = keras.models.Model(inputs=[input1, input2], outputs=out)
adam = keras.optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# checkpoint = ModelCheckpoint('/media/senilab/DATA2/new_feature_att/checkpoint/model-rgb', monitor='loss', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]
mc = ModelCheckpoint('/media/senilab/DATA2/new_feature_att_hmdb/checkpoint/model-weights-split1-b2-att128-0.000005-{epoch:08d}.h5', 
                                     save_weights_only=True, period=1000)


In [ ]:
model.load_weights('/media/senilab/DATA2/new_feature_att_hmdb/checkpoint/model-weights-split1-b2-att128-00013000-81.6.h5') #781

In [ ]:
model.evaluate(([test_feat_att_rgb, test_feat_att]),  test_label_att)

In [ ]:
model.fit([input_feat_att_rgb, input_feat_att], input_label_att_rgb, epochs=100000, callbacks=[mc])

In [ ]:
history = model.fit([input_feat_att_rgb, input_feat_att, input_feat_att_dp], input_label_att_rgb, epochs=20000, callbacks=[mc])

In [ ]:
test_out = model.predict(([test_feat_att_rgb, test_feat_att]))

In [ ]:
label_test_out = np.array([])
for i in range (len(test_feat_att_rgb)):
    if label_test_out.shape[0] == 0:
        test_label = test_out[i].argmax()
        label_test_out = np.array([test_label])
    else:
        test_label = test_out[i].argmax()
#         label_test_out = np.concatenate(([label_test_out],[test_label]))
        try:
            label_test_out = np.concatenate(([label_test_out],np.array([test_label])),axis=0)
        except:
            label_test_out = np.concatenate((label_test_out,np.array([test_label])),axis=0)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(test_label_att_rgb, label_test_out, labels=np.unique(test_label_att))
matrix

In [ ]:
plt.subplots(figsize=(20, 20))
 
sn.heatmap(matrix,annot=True,fmt='g',cmap="Blues")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax,cmap="Blues")
    plt.savefig(filename)

In [ ]:
cm_analysis(test_label_att, label_test_out, 'Matrix',labels=np.unique(test_label_att), figsize=(40,30))